<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow

In [ ]:
! pip install -U tf2onnx

In [ ]:
! python -m tf2onnx.convert --saved-model "/content/modelDir/model/1602656296" --output model.onnx

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output

In [ ]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [ ]:
LABEL = "survived"
feature_columns = []
NUMERIC_COLUMNS = ['age', 'fare']
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 
                       'embark_town', 'alone']

for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [ ]:
def make_train_input_fn(df, num_epochs):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [ ]:
def make_prediction_input_fn(df):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
# Instantiate the pre-made estimator
model = tf.estimator.LinearClassifier(feature_columns)

# Train the model
model.train(make_train_input_fn(dftrain, num_epochs=10))

In [ ]:
predDicts = list(model.predict(make_prediction_input_fn(dfeval)))

In [ ]:
preds = []
for pred in predDicts[:10]:
    preds.append(np.argmax(pred["probabilities"]))
preds

[0, 0, 1, 1, 0, 1, 1, 0, 1, 1]

In [ ]:
inputFn = \
tf.estimator.export.build_parsing_serving_input_receiver_fn(
    tf.feature_column.make_parse_example_spec(feature_columns)
)

OUTDIR = 'modelDir'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

modelBasePath = os.path.join(OUTDIR, "model")
modelPath = model.export_saved_model(modelBasePath, inputFn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/tmpytjlp56q/model.ckpt-49
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: modelDir/model/temp-1602656296/saved_model.pb


# Keras

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
df = pd.read_csv(URL)
df.pop("thal")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,0


In [ ]:
train = df.sample(frac=0.8, random_state=2).reset_index()
test = df.drop(train.index).reset_index()

In [ ]:
yTrain = train.pop("target")
yTest = test.pop("target")

In [ ]:
trainStats = train.describe().transpose()

def normalize(x):
    return (x - trainStats["mean"]) / trainStats["std"]

xTrain = normalize(train)
xTest = normalize(test)

In [ ]:
model = keras.Sequential(
    [
     layers.Dense(units=128, activation="relu", name="layer1"),
     layers.Dense(units=128, activation="relu", name="layer2"),
     layers.Dense(units=1, name="outputLayer")
    ]
)

In [ ]:
model.compile(
    optimizer="adam",
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    x=xTrain,
    y=yTrain,
    validation_data=(xTest, yTest),
    epochs=5
)   

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

8/8 [==============================] - 0s 21ms/step - loss: 0.9162 - accuracy: 0.7975 - val_loss: 0.8683 - val_accuracy: 0.8525
Epoch 2/5
8/8 [==============================] - 0s 4ms/step - loss: 0.7375 - accuracy: 0.8554 - val_loss: 1.0329 - val_accuracy: 0.7869
Epoch 3/5
8/8 [==============================] - 0s 4ms/step - loss: 0.5510 - accuracy: 0.8719 - val_loss: 0.7693 - val_accuracy: 0.8689
Epoch 4/5
8/8 [==============================] - 0s 4ms/step - loss: 0.4020 - accuracy: 0.8760 -

In [ ]:
res = pd.DataFrame(history.history)
res["epochs"] = history.epoch
res

,loss,accuracy,val_loss,val_accuracy,epochs
0,0.916225,0.797521,0.868335,0.852459,0
1,0.737518,0.855372,1.032906,0.786885,1
2,0.551038,0.871901,0.769302,0.868852,2
3,0.402040,0.876033,0.570993,0.885246,3
4,0.335513,0.888430,0.490023,0.868852,4


In [ ]:
loss, accuracy = model.evaluate(xTest, yTest, verbose=2)

2/2 - 0s - loss: 0.4900 - accuracy: 0.8689


In [ ]:
preds = model.predict(xTest)

In [ ]:
model.save("binaryClassification.h5")

In [ ]:
loadedModel = keras.models.load_model("/content/binaryClassification.h5")

In [ ]:
loadedModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 128)               1792      
_________________________________________________________________
layer2 (Dense)               (None, 128)               16512     
_________________________________________________________________
outputLayer (Dense)          (None, 1)                 129       
Total params: 18,433
Trainable params: 18,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
(loadedModel.predict(xTest) > 0.5).astype("int32").flatten()

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1], dtype=int32)

In [ ]:
! pip install keras2onnx

     |████████████████████████████████| 102kB 4.0MB/s 
     |████████████████████████████████| 71kB 4.2MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=0b925a775864e4e9105239e5e7e34d252c39e51afe1a4f4c02d4f16a0ecda6f6
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
import onnx
import keras2onnx

In [ ]:
model = load_model('/content/binaryClassification.h5')

In [ ]:
onnx_model_name = 'binaryClassification.onnx'
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, onnx_model_name)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 13 -> 8
The maximum opset needed by this model is only 9.


In [ ]:
! pip install --upgrade onnx2keras

  Created wheel for onnx2keras: filename=onnx2keras-0.0.23-cp36-none-any.whl size=24159 sha256=fdefc89a37142d57d2bf792dfb084b695e7f15853b5eb1800415360c958b5198
  Stored in directory: /root/.cache/pip/wheels/a4/d1/5a/644cd9e957d01977c2e7c5bcb75d8172ba0eab831e2471128d
Successfully built onnx2keras


In [ ]:
import onnx2keras
from onnx2keras import onnx_to_keras

In [ ]:
onnx_model = onnx.load('/content/binaryClassification.onnx')
k_model = onnx_to_keras(onnx_model, ['input_1'], name_policy='renumerate')

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> layer1_input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> outputLayer.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight outputLayer/kernel:0 with shape (128, 1).
DEBUG:onnx2keras:Found weight outputLayer/bias:0 with shape (1,).
DEBUG:onnx2keras:Found weight layer2/kernel:0 with shape (128, 128).
DEBUG:onnx2keras:Found weight layer2/bias:0 with shape (128,).
DEBUG:onnx2keras:Found weight layer1/kernel:0 with shape (13, 128).
DEBUG:onnx2keras:Found weight layer1/bias:0 with shape (128,).
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: MatMul
DEBUG:onnx2keras:node_name: layer10
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are availab

AttributeError: ignored

In [ ]:
import caffe2.python.onnx.backend

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [ ]:
model = onnx.load("/content/binaryClassification.onnx")

# prepare the caffe2 backend for executing the model this converts the ONNX model into a 
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be 
# availiable soon.
prepared_backend = caffe2.python.onnx.backend.prepare(model)

RuntimeError: ignored

In [ ]:
! pip install onnx

In [ ]:
# Some standard imports
import io
import numpy as np

from torch import nn
from torch.autograd import Variable
import torch.utils.model_zoo as model_zoo
import torch.onnx

import onnx
import caffe2.python.onnx.backend

In [ ]:
# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.


In [ ]:
# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
torch_model.load_state_dict(model_zoo.load_url(model_url))

# set the train mode to false since we will only run the forward pass.
torch_model.train(False)

In [ ]:
# Input to the model
x = Variable(torch.randn(batch_size, 1, 224, 224), requires_grad=True)

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run 
                               x,                       # model input (or a tuple for multiple inputs) 
                               "super_resolution.onnx", # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file

In [ ]:
# Load the ONNX GraphProto object. Graph is a standard Python protobuf object
model = onnx.load("/content/super_resolution.onnx")

In [ ]:
# prepare the caffe2 backend for executing the model this converts the ONNX model into a 
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be 
# availiable soon.
prepared_backend = caffe2.python.onnx.backend.prepare(model)

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph of the model itself contains inputs for all weight parameters, after the input image.
# Since the weights are already embedded, we just need to pass the input image.
# Set the first input
W = {model.graph.input[0].name: x.data.numpy()}

# Run the Caffe2 net:
c2_out = prepared_backend.run(W)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

print("Exported model has been executed on Caffe2 backend, and the result looks good!")

IndexError: ignored

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo


__all__ = ['SqueezeNet', 'squeezenet1_0', 'squeezenet1_1']


model_urls = {
    'squeezenet1_0': 'https://download.pytorch.org/models/squeezenet1_0-a815701f.pth',
    'squeezenet1_1': 'https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth',
}


class Fire(nn.Module):

    def __init__(self, inplanes, squeeze_planes,
                 expand1x1_planes, expand3x3_planes):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


class SqueezeNet(nn.Module):

    def __init__(self, version=1.0, num_classes=1000):
        super(SqueezeNet, self).__init__()
        if version not in [1.0, 1.1]:
            raise ValueError("Unsupported SqueezeNet version {version}:"
                             "1.0 or 1.1 expected".format(version=version))
        self.num_classes = num_classes
        if version == 1.0:
            self.features = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=False),
                Fire(96, 16, 64, 64),
                Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=False),
                Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=False),
                Fire(512, 64, 256, 256),
            )
        else:
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=False),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=False),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=False),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        # Final convolution is initialized differently form the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AvgPool2d(13)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal(m.weight.data, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x.view(x.size(0), self.num_classes)


def squeezenet1_1(pretrained=False, **kwargs):
    r"""SqueezeNet 1.1 model from the `official SqueezeNet repo
    <https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1>`_.
    SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters
    than SqueezeNet 1.0, without sacrificing accuracy.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = SqueezeNet(version=1.1, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['squeezenet1_1']))
    return model

In [ ]:
# Get pretrained squeezenet model
torch_model = squeezenet1_1(True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
Downloading: "https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-f364aa15.pth


In [ ]:
from torch.autograd import Variable
batch_size = 1    # just a random number

# Input to the model
x = Variable(torch.randn(batch_size, 3, 224, 224), requires_grad=True)

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                               x,                       # model input (or a tuple for multiple inputs)
                               "squeezenet.onnx",       # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file

/usr/local/lib/python3.6/dist-packages/torch/onnx/utils.py:44: UserWarning: You are exporting the model to ONNX while in training mode with 'train' parameter not specified. The model will default to inference mode export. If you wish to export a training amenable ONNX model, specify training=TrainingMode.TRAINING or training=TrainingMode.PRESERVE (to preserve the original model state) in torch.onnx.export().
  warnings.warn("You are exporting the model to ONNX while in training mode with "


In [ ]:
import caffe2.python.onnx.backend
from onnx import helper

# Load the ONNX GraphProto object. Graph is a standard Python protobuf object
model = onnx.load("/content/squeezenet.onnx")

# prepare the caffe2 backend for executing the model this converts the ONNX graph into a
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be
# availiable soon.
prepared_backend = caffe2.python.onnx.backend.prepare(model)

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph itself contains inputs for all weight parameters, followed by the input image.
# Since the weights are already embedded, we just need to pass the input image.
# last input the grap
W = {model.graph.input[0].name: x.data.numpy()}

# Run the Caffe2 net:
c2_out = prepared_backend.run(W)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

IndexError: ignored